## Model explanation using Shap values & plots

### Shapley values: a brief introduction

Shap values are generated using the [SHAP library](https://shap.readthedocs.io/en/latest/index.html) and are approximations of [Shapley Values](https://en.wikipedia.org/wiki/Shapley_value), a concept derived from game-theory. Briefly speaking, the value for a feature indicates, for every model decision passed to the explainer, how the model decision would be impacted by removing that feature. For a more in-depth explanation, please refer to this [summary article](https://towardsdatascience.com/understanding-how-ime-shapley-values-explains-predictions-d75c0fceca5a).

By default, RSMExplain uses the [Sampling](https://shap.readthedocs.io/en/latest/generated/shap.explainers.Sampling.html#shap.explainers.Sampling) explainer which computes shap values through random permutations of the features, a method described [here](https://link.springer.com/article/10.1007/s10115-013-0679-x).

The sampling explainer is model agnostic and should, in principle, work for any type of model. RSMExplain currently only supports regressors. 


### Reading Shap values

Shap values are additive representations of a feature's impact on a model decision. The sum of all shap values and the base value for a example yields the actual model prediction for that example.

A shap value for a feature can be considered that feature's contribution to that specific prediction. By computing the averge of all absolute shap values for a specific feature, we can calculate that feature's average impact on the predictions for the data we are trying to explain. The average, mximum, and minimum absolute shap values for all features can be found in the `absolute_shap_values.csv` file in the `output` directory.



### Things to consider

- RSMExplain can only generate shap values for the examples contained in "explainable_data" and, if specified, "sample_range" or "sample_size". If the dataset passed is small, then the values derived may not be representative of the model as a whole. Plots displaying mean values for your shap values may be unreliable if your dataset was small or not actually representative of the data the model deals with.

- As long as a sufficiently large background set was passed, the shap values for individual examples may be considered reliable.

- If you wish to analyze shap values manually, please refer to the `*shap*.csv` files in the `output` directory.

- If you wish to use the generated shap explanation object for additional processing, you may unpickle `explanation.pkl`. If you chose specific examples via `sample_range` or `sample_size`, you can find them in the `ids.pkl` file where they are stored as a mapping between the position of the example in the dataset and the ID of the example.

### Shap values summary

This is a quick textual summary of your shap values. Please refer to the Plots section below for visualizations.

All values are rounded to $10^{-3}$ unless specified otherwise.

#### Top 5 features by mean absolute Shap value

The following table shows the top 5 features in terms of mean absolute shap value, i.e., the top 5 features with the biggest mean impact on model predictions. Note that the table also includes the maximum and minimum absolute values for each feature.

In [ ]:
if len(df_abs_shap) > 5:
    display(Markdown("Your model has more than 5 features. Displaying the top 5:"))
    top_5_table = HTML(df_abs_shap[0:5].to_html(classes=['sortable'], index=False, float_format=float_format_func))
else:
    display(Markdown("Your model has 5 or fewer features. Displaying all:"))
    top_5_table = HTML(df_abs_shap.to_html(classes=['sortable'], index=False, float_format=float_format_func))

display(top_5_table)

In [ ]:
mean_values = df_abs_shap["abs. mean shap"]
rows_with_zero_mean_values = df_abs_shap[df_abs_shap["abs. mean shap"] == 0]

msg = ("#### Features with zero mean Shap value\n The featuers in table below "
       "likely did not contribute to the model decisions. Assuming the examples passed "
       "were sufficient in number and representative of the data the model usually "
       "encounters, the features in this table are not useful to the model.\n "
       "**IMPORTANT**: Before you draw this conclusion, please make sure that those "
       "features were not simply 0 in all the examples you provided the model.")

if len(rows_with_zero_mean_values) > 0:
    display(Markdown(msg))
    
    if len(rows_with_zero_mean_values) <= 10:
        zero_value_table = HTML(rows_with_zero_mean_values.to_html(classes=['sortable'], 
                                                        index=False, 
                                                        float_format=float_format_func))
    else:
        display(Markdown("You have more than 10 features with absolute mean shap value of 0."
                         "Displaying the first 10 here. Check `absolute_shap_values.csv` for the rest."))
        zero_value_table = HTML(rows_with_zero_mean_values[:10].to_html(classes=['sortable'], 
                                                                       index=False, 
                                                                       float_format=float_format_func))
    display(zero_value_table)

#### Bottom 5 features by mean absolute Shap value

The following features are the ones with the lowest absolute non-zero mean shap value. Assuming that your dataset was large enough and representative, these features may be the least useful to the model.
Note that the table also includes the maximum and minimum abs. shap value for each feature as well. 

*Rounding is disabled for this table to prevent values appearing as 0.*

In [ ]:
mean_abs_nonzero_values = df_abs_shap[df_abs_shap['abs. mean shap'] != 0]
if len(df_abs_shap) > 5:
    display(Markdown("Your model has more than 5 features. Displaying the bottom 5:"))
    bottom_5_table = HTML(mean_abs_nonzero_values[-5:-1].to_html(index=False, classes=['sortable']))
else:
    display(Markdown("Your model has 5 or less features. Displaying all:"))
    bottom_5_table = HTML(mean_abs_nonzero_values.to_html(index=False, classes=['sortable']))

display(bottom_5_table)

#### Top 5 features by maximum absolute Shap value

Features in the table below are the ones with the largest impact according to the maximum absolute Shap value. If these *do not* overlap with the features with the largest mean impact, then it is likely that they have large outlier values, but lower average impact. 

In [ ]:
if len(df_abs_shap) > 5:
    display(Markdown("Your model has more than 5 features. Displaying the top 5:"))
    top_5_max_table = HTML(df_abs_shap.sort_values(by=['abs. max shap'], 
                                                   ascending=False)[0:5].to_html(classes=['sortable'], 
                                                                                 float_format=float_format_func,
                                                                                 index=False))
else:
    display(Markdown("Your model has 5 or fewer features. Displaying all:"))
    top_5_max_table = HTML(df_abs_shap.sort_values(by=['abs. max shap'], 
                                                   ascending=False).to_html(index=False, 
                                                                            classes=['sortable'],
                                                                            float_format=float_format_func))
    
display(top_5_max_table)